In [ ]:
!pip install -q transformers openpyxl pandas

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd

# Load pretrained model & tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Load your Excel file (uploaded earlier)
df = pd.read_excel("TrainVersion.xlsx")

# Preview
df.head()


,text,Sentiment Label
0,"Phoebe, Your videos are amazing. Also you have...",NaN
1,Malaysia's weather was unbearable and humid al...,NaN
2,"Thank you for visiting Malaysia, I am proud of...",NaN
3,Is it one of your bucket list to visit buc-ket...,NaN
4,For mother Russia have a good trip in Malaysia...,NaN


In [ ]:
def get_sentiment_label(text):
    try:
        result = sentiment_pipeline(text[:512])[0]  # Truncate long comments
        label = result['label']
        if label == 'LABEL_2':
            return 1
        elif label == 'LABEL_1':
            return 0
        else:
            return -1
    except:
        return None

# Apply sentiment labeling
df['Sentiment Label'] = df['text'].astype(str).apply(get_sentiment_label)

# Show a few results
df[['text', 'Sentiment Label']].head()


,text,Sentiment Label
0,"Phoebe, Your videos are amazing. Also you have...",1
1,Malaysia's weather was unbearable and humid al...,-1
2,"Thank you for visiting Malaysia, I am proud of...",1
3,Is it one of your bucket list to visit buc-ket...,0
4,For mother Russia have a good trip in Malaysia...,1


In [ ]:
df[['text', 'Sentiment Label']].head(20)


,text,Sentiment Label
0,"Phoebe, Your videos are amazing. Also you have...",1
1,Malaysia's weather was unbearable and humid al...,-1
2,"Thank you for visiting Malaysia, I am proud of...",1
3,Is it one of your bucket list to visit buc-ket...,0
4,For mother Russia have a good trip in Malaysia...,1
5,"Once again Niki, I had to go looking for your ...",1
6,The food made me sick during my trip to Malaysia.,-1
7,Central market back in those days use to be a ...,0
8,This video is amazing! The content is very wel...,1
9,Disappointed by the pollution in some parts of...,-1


In [ ]:
df['Sentiment Label'].value_counts()

,count
Sentiment Label,
1,1510
-1,1164
0,855


In [ ]:
df.to_excel("TrainVersion_labeled.xlsx", index=False)